In [ ]:
%run yulesimon-common.ipynb

# Trials

In [ ]:
def trial(num_alphas=100, random_alpha=False):
    print('Processing...')
    print()

    a_N = np.array([])
    a_sqrt_mse = np.array([])

    # 10 random states
    # random_states = [0, 3, 5, 8, 11, 16, 17, 20, 21, 24]
    random_states = [3]
    # random_states = [0, 3, 5]
    # random_states = [3, 5, 8]
    # random_states = [0, 3, 17]

    # hold array of absolute errors
    abs_errors = np.array([], dtype=float)

    # change N [32..2048]
    # for i in range(5, 12):
    for i in range(5, 8):
    # for i in range(5, 12):

        N = 2**i

        a_N = np.append(a_N, N)

        # data generation (fix random state=0)
        DATA_SEED = 0
        fix_random(seed=DATA_SEED)
        X, y, M = generate_data(N=N, num_alphas=num_alphas, samples_per_alpha=100, random_alpha=random_alpha, random_state=DATA_SEED)

        # reshape X for Conv1D
        X = X.reshape(X.shape[0], X.shape[1], 1)

        # split train/test
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

        avg_sqrt_mse = 0

        # change random state
        for rs in random_states:

            # fix random
            fix_random(seed=rs)
        
            # training
            model, history = train(X_train, y_train, filters=32, batch_size=32, random_state=rs)

            # predict
            y_pred = model.predict(X_test).flatten()

            # mse
            mse = mean_squared_error(y_test, y_pred)

            # sqrt_mse
            sqrt_mse = np.sqrt(mse)

            # avg_sqrt_mse (accumulator)
            avg_sqrt_mse += sqrt_mse
            
            print('N = {}, M = {}, random_state = {}    =>    sqrt_mse = {:.6f}'.format(N, M, rs, sqrt_mse))

            # absolute errors
            abs_errors = np.append(abs_errors, np.abs(y_test - y_pred))

        # avg_sqrt_mse
        avg_sqrt_mse = avg_sqrt_mse / len(random_states)
        a_sqrt_mse = np.append(a_sqrt_mse, avg_sqrt_mse)
        print('avg_sqrt_mse = {:.6f}'.format(avg_sqrt_mse))
        print()

    return abs_errors, a_N, a_sqrt_mse

In [ ]:
abs_errors, a_N, a_sqrt_mse = trial(random_alpha=True)

# plot log(N) vs sqrt_mse

In [ ]:
def plot_sqrt_mse():
    
    plt.figure(figsize=(15, 4))
    
    ax1 = plt.subplot(1, 2, 1)
    ax1.set(title='sqrt_mse', xlabel='log(N)', ylabel='sqrt_mse')
    ax1.scatter(np.log10(a_N), a_sqrt_mse)

plot_sqrt_mse()

# plot abs(error) distributions

In [ ]:
_ = sns.distplot(abs_errors, hist=False, kde=True, kde_kws={"shade": True})

# avg & std of errors

In [ ]:
print('avg(abs_errors):\t{:.6f}'.format(np.average(abs_errors)))
print()
print('std(abs_errors):\t{:.6f}'.format(np.std(abs_errors)))